In [6]:
import pandas as pd
from rapidfuzz import fuzz

# Carregar dataset
df = pd.read_csv(
    "Transactions_Tabela1.csv",
    sep=";",
    skiprows=1,
    decimal=",",
    encoding="utf-8"
)

print("Total de registros iniciais:", len(df))

# Remover duplicatas exatas
df = df.drop_duplicates()

# Garantir strings
df["brand"] = df["brand"].astype(str)
df["product_line"] = df["product_line"].astype(str)

# Criar chave aproximada para reduzir comparações
df["block_key"] = (
    df["brand"].str[0] +
    df["product_line"].str[0] +
    df["product_line"].str.len().astype(str)
)

# Ordenar dentro de cada bloco
df = df.sort_values(["block_key", "brand", "product_line"]).reset_index(drop=True)

remover = set()

# Comparar somente a linha atual com a próxima
for i in range(len(df) - 1):
    if df.loc[i, "block_key"] == df.loc[i+1, "block_key"]:
        score = fuzz.ratio(df.loc[i, "product_line"], df.loc[i+1, "product_line"])
        if score > 92:  # limiar recomendado
            # manter a mais recente
            data_i = pd.to_datetime(df.loc[i, "transaction_date"], dayfirst=True)
            data_j = pd.to_datetime(df.loc[i+1, "transaction_date"], dayfirst=True)

            if data_i >= data_j:
                remover.add(i+1)
            else:
                remover.add(i)

print("Duplicatas aproximadas detectadas:", len(remover))

df_final = df.drop(list(remover))

print("Total final:", len(df_final))

df_final.to_csv("Transactions_Limpo.csv", index=False)
print("Arquivo salvo como Transactions_Limpo.csv")


Total de registros iniciais: 20000
Duplicatas aproximadas detectadas: 13312
Total final: 6688
Arquivo salvo como Transactions_Limpo.csv


# usei o RapidFuzz porque o fuzzywuzzy dava 10 minutos e executava o arquivo 